# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
491,YUM
492,ZBH
493,ZBRA
494,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
from my_secrets2 import IEX_CLOUD_API_TOKEN


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
symbol='AAPL'
stats_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
stats_response = requests.get(stats_url, verify=True)
stats_data = stats_response.json()
print(stats_data)


{'companyName': 'Apple Inc', 'marketcap': 2933256545780, 'week52high': 198.57, 'week52low': 163.63, 'week52highSplitAdjustOnly': 199.62, 'week52lowSplitAdjustOnly': 164.08, 'week52change': 0.0735013687437076, 'sharesOutstanding': 15334082000, 'float': 0, 'avg10Volume': 46212909, 'avg30Volume': 60551436, 'day200MovingAvg': 182.47, 'day50MovingAvg': 179.42, 'employees': 164000, 'ttmEPS': 6.43, 'ttmDividendRate': 0.966204324874062, 'dividendYield': 0.005050992340812703, 'nextDividendDate': '', 'exDividendDate': '2024-05-10', 'nextEarningsDate': '2024-08-01', 'peRatio': 29.21890392154519, 'beta': 1.0518841825889087, 'maxChangePercent': 74.16306483300589, 'year5ChangePercent': 3.475865796560914, 'year2ChangePercent': 0.30130136939707075, 'year1ChangePercent': 0.0903946203626591, 'ytdChangePercent': -0.0012003974521708205, 'month6ChangePercent': 0.01546690562087405, 'month3ChangePercent': 0.05726555720752336, 'month1ChangePercent': 0.10552827711199896, 'day30ChangePercent': 0.126103055884240

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [9]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url=f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        new_series = pd.Series(
            [symbol, data[symbol]['price'], data[symbol]['stats'].get('year1ChangePercent', 'N/A'), 'N/A'],
            index=my_columns
        )

        # Use pd.concat to append the new series to the DataFrame
        final_dataframe = pd.concat([final_dataframe, new_series.to_frame().T], ignore_index=True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,130.41,0.124389,N/A
1,AAL,11.5,-0.21751,N/A
2,AAP,70.64,-0.393735,N/A
3,AAPL,192.25,0.090395,N/A
4,ABBV,161.24,0.192135,N/A
...,...,...,...,...
491,YUM,137.43,0.075717,N/A
492,ZBH,115.15,-0.087837,N/A
493,ZBRA,312.34,0.182389,N/A
494,ZION,43.19,0.540739,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [12]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,270,LB,79.92,2.281287,N/A
1,204,GPS,28.96,1.799811,N/A
2,339,NVDA,1096.33,1.755868,N/A
3,334,NRG,81,1.566925,N/A
4,406,SLG,52.97,1.422858,N/A
5,194,GE,165.14,1.021926,N/A
6,439,TT,327.46,0.979697,N/A
7,480,WRK,53.64,0.952259,N/A
8,228,HWM,84.65,0.943672,N/A
9,455,URI,669.41,0.924001,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [13]:
portfolio_size = input('Enter your portfolio size: ')
val = float(portfolio_size)
print(val)

10000000.0


In [14]:
"""position_size=val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number Of Shares To Buy']=math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe""" 
# Assuming final_dataframe is already defined
position_size = val / len(final_dataframe.index)

# Use vectorized operations to calculate the 'Number Of Shares To Buy'
final_dataframe['Number of Shares to Buy'] = (position_size / final_dataframe['Price']).apply(math.floor)

final_dataframe

C:\Users\iamha\AppData\Local\Temp\ipykernel_18932\533946582.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe['Number of Shares to Buy'] = (position_size / final_dataframe['Price']).apply(math.floor)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,270,LB,79.92,2.281287,2502
1,204,GPS,28.96,1.799811,6906
2,339,NVDA,1096.33,1.755868,182
3,334,NRG,81,1.566925,2469
4,406,SLG,52.97,1.422858,3775
5,194,GE,165.14,1.021926,1211
6,439,TT,327.46,0.979697,610
7,480,WRK,53.64,0.952259,3728
8,228,HWM,84.65,0.943672,2362
9,455,URI,669.41,0.924001,298


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [7]:

hqm_columns = ['Ticker', 'Price','Number of Shares to Buy', 'One-Year Price Return', 'One-Year Return Percentile','Six-Month Price Return','Six-Month Return Percentile','Three-Month Price Return','Three-Month Return Percentile','One-Month Price Return','One-Month Return Percentile','HQM Score']
hqm_dataframe = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
    batch_api_url=f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        new_series = pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats'].get('year1ChangePercent', 'N/A'),
                'N/A',
                data[symbol]['stats'].get('month6ChangePercent', 'N/A'),
                'N/A',
                data[symbol]['stats'].get('month3ChangePercent', 'N/A'),
                'N/A',
                data[symbol]['stats'].get('month1ChangePercent', 'N/A'),
                'N/A',
                'N/A'
            ],
            index=hqm_columns
        )

        # use pd.concat to append the new series to the DataFrame
        hqm_dataframe = pd.concat([hqm_dataframe, new_series.to_frame().T], ignore_index=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.41,N/A,0.124389,N/A,0.033436,N/A,-0.039242,N/A,-0.058529,N/A,N/A
1,AAL,11.5,N/A,-0.21751,N/A,-0.064595,N/A,-0.260026,N/A,-0.181688,N/A,N/A
2,AAP,70.64,N/A,-0.393735,N/A,0.307715,N/A,0.011373,N/A,-0.112172,N/A,N/A
3,AAPL,192.25,N/A,0.090395,N/A,0.015467,N/A,0.057266,N/A,0.105528,N/A,N/A
4,ABBV,161.24,N/A,0.192135,N/A,0.150016,N/A,-0.113653,N/A,-0.032256,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,137.43,N/A,0.075717,N/A,0.084933,N/A,-0.022347,N/A,-0.053687,N/A,N/A
492,ZBH,115.15,N/A,-0.087837,N/A,0.008461,N/A,-0.07633,N/A,-0.059732,N/A,N/A
493,ZBRA,312.34,N/A,0.182389,N/A,0.338095,N/A,0.16063,N/A,0.055365,N/A,N/A
494,ZION,43.19,N/A,0.540739,N/A,0.227489,N/A,0.130485,N/A,0.041841,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [9]:
time_periods=[
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]
problematic_rows = []
for row in hqm_dataframe.index:
    for time_period in time_periods:
        price_return = hqm_dataframe.loc[row, f'{time_period} Price Return']
        
        # Check if the price_return is NaN
        if pd.isna(price_return):
            hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = np.nan
            problematic_rows.append((row, f'{time_period} Price Return'))
        else:
            # Calculate the percentile excluding NaN values
            valid_returns = hqm_dataframe[f'{time_period} Price Return'].dropna()
            return_percentile = stats.percentileofscore(valid_returns, price_return)
            hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = return_percentile

# Output the problematic rows
if problematic_rows:
    print("Problematic rows with NaN values in 'Price Return':")
    for row, time_period in problematic_rows:
        print(f"Row: {row}, Time Period: {time_period}")

hqm_dataframe

Problematic rows with NaN values in 'Price Return':
Row: 116, Time Period: One-Year Price Return
Row: 116, Time Period: Six-Month Price Return
Row: 116, Time Period: Three-Month Price Return
Row: 116, Time Period: One-Month Price Return
Row: 162, Time Period: One-Year Price Return
Row: 162, Time Period: Six-Month Price Return
Row: 162, Time Period: Three-Month Price Return
Row: 162, Time Period: One-Month Price Return
Row: 321, Time Period: One-Year Price Return
Row: 321, Time Period: Six-Month Price Return
Row: 321, Time Period: Three-Month Price Return
Row: 321, Time Period: One-Month Price Return
Row: 322, Time Period: One-Year Price Return
Row: 322, Time Period: Six-Month Price Return
Row: 322, Time Period: Three-Month Price Return
Row: 322, Time Period: One-Month Price Return


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.41,N/A,0.124389,45.325203,0.033436,34.146341,-0.039242,31.504065,-0.058529,19.715447,N/A
1,AAL,11.5,N/A,-0.21751,6.097561,-0.064595,12.601626,-0.260026,1.829268,-0.181688,2.03252,N/A
2,AAP,70.64,N/A,-0.393735,1.626016,0.307715,85.569106,0.011373,48.98374,-0.112172,6.707317,N/A
3,AAPL,192.25,N/A,0.090395,39.837398,0.015467,30.081301,0.057266,66.056911,0.105528,93.089431,N/A
4,ABBV,161.24,N/A,0.192135,56.910569,0.150016,63.821138,-0.113653,16.260163,-0.032256,33.739837,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,137.43,N/A,0.075717,37.398374,0.084933,47.764228,-0.022347,36.178862,-0.053687,22.154472,N/A
492,ZBH,115.15,N/A,-0.087837,14.634146,0.008461,28.04878,-0.07633,23.373984,-0.059732,18.495935,N/A
493,ZBRA,312.34,N/A,0.182389,55.284553,0.338095,88.821138,0.16063,88.821138,0.055365,82.520325,N/A
494,ZION,43.19,N/A,0.540739,90.04065,0.227489,76.219512,0.130485,84.756098,0.041841,79.471545,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentiles=[]
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score']=mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.41,N/A,0.124389,45.325203,0.033436,34.146341,-0.039242,31.504065,-0.058529,19.715447,32.672764
1,AAL,11.5,N/A,-0.21751,6.097561,-0.064595,12.601626,-0.260026,1.829268,-0.181688,2.03252,5.640244
2,AAP,70.64,N/A,-0.393735,1.626016,0.307715,85.569106,0.011373,48.98374,-0.112172,6.707317,35.721545
3,AAPL,192.25,N/A,0.090395,39.837398,0.015467,30.081301,0.057266,66.056911,0.105528,93.089431,57.26626
4,ABBV,161.24,N/A,0.192135,56.910569,0.150016,63.821138,-0.113653,16.260163,-0.032256,33.739837,42.682927
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,137.43,N/A,0.075717,37.398374,0.084933,47.764228,-0.022347,36.178862,-0.053687,22.154472,35.873984
492,ZBH,115.15,N/A,-0.087837,14.634146,0.008461,28.04878,-0.07633,23.373984,-0.059732,18.495935,21.138211
493,ZBRA,312.34,N/A,0.182389,55.284553,0.338095,88.821138,0.16063,88.821138,0.055365,82.520325,78.861789
494,ZION,43.19,N/A,0.540739,90.04065,0.227489,76.219512,0.130485,84.756098,0.041841,79.471545,82.621951


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score',ascending=False,inplace=True)
hqm_dataframe=hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,1096.33,N/A,1.755868,99.593496,1.295806,100.0,0.422948,99.593496,0.259159,99.390244,99.644309
1,NRG,81,N/A,1.566925,99.390244,0.830434,99.796748,0.555922,100.0,0.150899,96.95122,99.034553
2,HWM,84.65,N/A,0.943672,98.373984,0.635067,98.98374,0.262459,96.544715,0.255875,99.186992,98.272358
3,QCOM,204.05,N/A,0.872719,97.357724,0.630725,98.780488,0.324883,97.560976,0.220349,98.577236,98.069106
4,MU,125,N/A,0.780626,96.341463,0.652766,99.186992,0.409117,98.98374,0.10432,92.479675,96.747967
5,LB,79.92,N/A,2.281287,100.0,0.815831,99.593496,0.215421,95.121951,0.079271,87.804878,95.630081
6,WDC,75.29,N/A,0.890113,97.560976,0.595731,98.577236,0.314132,97.357724,0.080876,88.414634,95.477642
7,WRK,53.64,N/A,0.952259,98.577236,0.370978,92.073171,0.211628,94.512195,0.124012,95.731707,95.223577
8,LNC,32.99,N/A,0.607752,92.073171,0.397458,95.121951,0.213343,94.918699,0.162799,97.764228,94.969512
9,APH,132.37,N/A,0.749475,94.918699,0.481863,97.96748,0.230627,95.528455,0.089577,90.243902,94.664634


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_size = input('Enter your portfolio size: ')
val = float(portfolio_size)
print(val)

10000000.0


In [15]:

position_size = val / len(hqm_dataframe.index)

# Use vectorized operations to calculate the 'Number Of Shares To Buy'
hqm_dataframe['Number of Shares to Buy'] = (position_size / hqm_dataframe['Price']).apply(math.floor)

hqm_dataframe

C:\Users\iamha\AppData\Local\Temp\ipykernel_11840\1549653267.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe['Number of Shares to Buy'] = (position_size / hqm_dataframe['Price']).apply(math.floor)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,1096.33,182,1.755868,99.593496,1.295806,100.0,0.422948,99.593496,0.259159,99.390244,99.644309
1,NRG,81,2469,1.566925,99.390244,0.830434,99.796748,0.555922,100.0,0.150899,96.95122,99.034553
2,HWM,84.65,2362,0.943672,98.373984,0.635067,98.98374,0.262459,96.544715,0.255875,99.186992,98.272358
3,QCOM,204.05,980,0.872719,97.357724,0.630725,98.780488,0.324883,97.560976,0.220349,98.577236,98.069106
4,MU,125,1600,0.780626,96.341463,0.652766,99.186992,0.409117,98.98374,0.10432,92.479675,96.747967
5,LB,79.92,2502,2.281287,100.0,0.815831,99.593496,0.215421,95.121951,0.079271,87.804878,95.630081
6,WDC,75.29,2656,0.890113,97.560976,0.595731,98.577236,0.314132,97.357724,0.080876,88.414634,95.477642
7,WRK,53.64,3728,0.952259,98.577236,0.370978,92.073171,0.211628,94.512195,0.124012,95.731707,95.223577
8,LNC,32.99,6062,0.607752,92.073171,0.397458,95.121951,0.213343,94.918699,0.162799,97.764228,94.969512
9,APH,132.37,1510,0.749475,94.918699,0.481863,97.96748,0.230627,95.528455,0.089577,90.243902,94.664634


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
# Define the Excel writer and the sheet name
writer = pd.ExcelWriter('recommended_trades2.xlsx', engine='xlsxwriter')

# Write the DataFrame to an Excel sheet
hqm_dataframe.to_excel(writer, sheet_name='Recommended Trades2', index=False)

# Get the workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Recommended Trades2']

# Define formats
background_color = '#0a0a24'
font_color = '#ffffff'

string_format = workbook.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = workbook.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = workbook.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
percent_format = workbook.add_format(
    {
        'num_format': '0.00%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

# Apply formats to columns
for col_num, col_name in enumerate(hqm_columns):
    if col_name == 'Price':
        worksheet.set_column(col_num, col_num, 18, dollar_format)
    elif col_name == 'Number of Shares to Buy':
        worksheet.set_column(col_num, col_num, 18, integer_format)
    elif col_name == 'Ticker':
        worksheet.set_column(col_num, col_num, 18, string_format)
    else:
        worksheet.set_column(col_num, col_num, 18, percent_format)

# Close the Excel writer
writer.close()